# Open Class Rooms - Formation Ingénieur Machine Learning
## Projet : Définissez votre stratégie d'apprentissage

Created by: Thomas Durand-Texte, Decembre 2022

# Import des packages et chargemetn des données
Les données correspondent à un temps estimatif de travail des différents projets, plus un score de difficulté et de motivation (non utilisé pour le moment)

In [11]:
import pandas as pd
import numpy as np

data = pd.read_csv('projets.csv') ;
display( data )

,projet,duree,duree cours associes,temps ressourses additionneles,difficulté,motivation
0,Application santé,70h,25h,30h,3,5
1,Besoin conso batiements,100h,47h,40h,2,5
2,Segmentez clients e-commerce,70h,25h,20h,3,3
3,Catégorisez questions,80h,8h,80h,4,2
4,Classez images algo Deep Learning,100h,40h,60h,4,5
5,Dévelopez une preuve de concept,30h,1h,20h,4,3
6,Compétition Kaggle,50h,0h,50h,3,2


# Vérification du type des colones et conversion des temps/durée str->int
* Toutes les colonnes sont vérifiées avec une boucle for
* si la colonne est déjà en int, ou si 'duree' ou 'temps' n'est pas présent dans la key, alors la colonne est inchangée

In [12]:
for key in data.keys():
    if (data[key].dtype == 'int64') or ( not 'duree' in key) and ( not'temps' in key): continue ;
    data[key] = pd.to_numeric(data[key].str.replace('h',''), errors='coerce' ) ;
# for #

# Vérification du type des données et des données

In [8]:
print('dtypes:\n', data.dtypes)
display(data)

dtypes:
 projet                            object
duree                              int64
duree cours associes               int64
temps ressourses additionneles     int64
difficulté                         int64
motivation                         int64
dtype: object


,projet,duree,duree cours associes,temps ressourses additionneles,difficulté,motivation
0,Application santé,70,25,30,3,5
1,Besoin conso batiements,100,47,40,2,5
2,Segmentez clients e-commerce,70,25,20,3,3
3,Catégorisez questions,80,8,80,4,2
4,Classez images algo Deep Learning,100,40,60,4,5
5,Dévelopez une preuve de concept,30,1,20,4,3
6,Compétition Kaggle,50,0,50,3,2


# Calcul du temps total pour chaque projet

In [13]:
data['temps total'] = data['duree'] + data['duree cours associes'] + data['temps ressourses additionneles'] ;

display( data )        

,projet,duree,duree cours associes,temps ressourses additionneles,difficulté,motivation,temps total
0,Application santé,70,25,30,3,5,125
1,Besoin conso batiements,100,47,40,2,5,187
2,Segmentez clients e-commerce,70,25,20,3,3,115
3,Catégorisez questions,80,8,80,4,2,168
4,Classez images algo Deep Learning,100,40,60,4,5,200
5,Dévelopez une preuve de concept,30,1,20,4,3,51
6,Compétition Kaggle,50,0,50,3,2,100


# Temps initial estimatif en heures sur 6 mois
1. calcul du nombre de jours de janvier à juin inclu
1. calcul du nombre de semaine (entière)
1. calcul du nombre d'heures à disposition sur une base de 35h/semaine
1. sum des heures des différents projets (estimation) et comparaison avec e nombre d'heures à disposition

In [14]:
n_jours = np.sum( [31,28,31,30,31,30]) ;
n_sem = n_jours // 7 +2
n_jours_restant = n_jours - 7*n_sem
n_heures_dispo = 35 * n_sem
heures_total_estim = data['temps total'].sum() ;
print('nombre de jours {:}\nnombre de semaines {:}\nnombre jours restants {:}\nnombre heures disponibles {:}\nnombre heures total estimatif {:}'.format( n_jours, n_sem, n_jours_restant, n_heures_dispo, heures_total_estim) ) ;
print('nombre heures manquantes {:}'.format( heures_total_estim - n_heures_dispo))

nombre de jours 181
nombre de semaines 27
nombre jours restants -8
nombre heures disponibles 945
nombre heures total estimatif 946
nombre heures manquantes 1


# Estimation de la durée de chaque projet
arrondi et ceil

In [18]:
data['duree estimatif (semaines)'] = np.round( data['temps total'] / 35 , 2 )
data['duree estimatif ceil (semaines)'] = np.ceil( data['temps total'] / 35 )
print('n semaines initial (janvier à juin) : {:}'.format( n_sem ) )
print('n semaines ceil : {:}'.format( data['duree estimatif ceil (semaines)'].sum() ) )
display( data )

n semaines initial (janvier à juin) : 27
n semaines ceil : 30.0


,projet,duree,duree cours associes,temps ressourses additionneles,difficulté,motivation,temps total,duree estimatif (semaines),duree estimatif ceil (semaines)
0,Application santé,70,25,30,3,5,125,3.57,4.0
1,Besoin conso batiements,100,47,40,2,5,187,5.34,6.0
2,Segmentez clients e-commerce,70,25,20,3,3,115,3.29,4.0
3,Catégorisez questions,80,8,80,4,2,168,4.80,5.0
4,Classez images algo Deep Learning,100,40,60,4,5,200,5.71,6.0
5,Dévelopez une preuve de concept,30,1,20,4,3,51,1.46,2.0
6,Compétition Kaggle,50,0,50,3,2,100,2.86,3.0


# Estimation des dates de soutenance
1. Conversion des durées cumulées (semaines + ceil) en *timedelta*
1. Ajout à une date initiale le 02/01/2023

In [22]:
data['date soutenance (estimation)'] = pd.to_datetime( '02/01/2023',format='%d/%m/%Y') + pd.to_timedelta( (7*data['duree estimatif ceil (semaines)'].cumsum()).astype(str) + ' days')
display(data)

,projet,duree,duree cours associes,temps ressourses additionneles,difficulté,motivation,temps total,duree estimatif (semaines),duree estimatif ceil (semaines),date soutenance (estimation)
0,Application santé,70,25,30,3,5,125,3.57,4.0,2023-01-30
1,Besoin conso batiements,100,47,40,2,5,187,5.34,6.0,2023-03-13
2,Segmentez clients e-commerce,70,25,20,3,3,115,3.29,4.0,2023-04-10
3,Catégorisez questions,80,8,80,4,2,168,4.80,5.0,2023-05-15
4,Classez images algo Deep Learning,100,40,60,4,5,200,5.71,6.0,2023-06-26
5,Dévelopez une preuve de concept,30,1,20,4,3,51,1.46,2.0,2023-07-10
6,Compétition Kaggle,50,0,50,3,2,100,2.86,3.0,2023-07-31


# Création de la chaine de caractère à copier/coller dans la fiche projet

In [23]:
str_out = '' ;
for i, date in enumerate( data['date soutenance (estimation)'].tolist() ):
    date_str = date.strftime('%d/%m/%Y') ; 
    # print('date:', date_str )
    str_out += '\n-Projet {:}: {:}'.format( i+1, date_str )
print(str_out)


-Projet 1: 30/01/2023
-Projet 2: 13/03/2023
-Projet 3: 10/04/2023
-Projet 4: 15/05/2023
-Projet 5: 26/06/2023
-Projet 6: 10/07/2023
-Projet 7: 31/07/2023
